In [ ]:
import pandas as pd
import requests
import json
from pyjstat import pyjstat

## Først et lite sports-api
* [https://api-sports.io/](https://api-sports.io/)
* Lag gratis konto eller logg in med google

In [ ]:
api_key = "31d5cf4a12d29e845d6b08db3ede0685" #Din api-nøkkel

headers = {"x-apisports-key": api_key}

base_url = "https://v3.football.api-sports.io/"



# SSB sitt webAPI

* SSB har et eget webAPI slik at du automatisk kan hente inn nyeste data til:
  * Analyse
  * Visualisering
* Man kan få data på flere formater, vi anbefaler `json_stat`
  * Bruk `pyjstat` til å lese data inn i dataframe slik som fra eurostat

* Dersom man prøver å hente inn data fra feks tabell 09695 med et GET request får man tilbake meta-data om tabellen:
* [https://data.ssb.no/api/v0/no/table/09695](https://data.ssb.no/api/v0/no/table/09695)

In [ ]:
konkurser_url ="https://data.ssb.no/api/v0/no/table/09695"
response = requests.get(konkurser_url)
metadata = response.json()
metadata

* Det kan være nyttig å hente ut metadata og undersøke den
* Vi kan laste det inn i python og gjøre filtrering på hva vi vil ha med
* Vi kan kan også "klikke på linken" i en nettleser og se på verdiene der

## Sende spørring mot SSB sine dataset
* For å hente inn de faktiske dataene må vi sende en POST request til samme URL
* Denne post-requesten må inneholde egen data om spørringen vi gjør (Hvilke variabler vil vi ha + eventuelle filtreringer

![img/ssbapikonsoll.png](img/ssbapikonsoll.png)

* Det er ikke nødvendigvis plankekjøring å lage denne spørringen:
  * Vi kan bruke SSB sin [API-konsoll](https://data.ssb.no/api/v0/no/console) (bildet over)
  * I statistikkbanken kan vi undersøke og velge ut data, og nederst på siden kan vi hente spørringen for dataene vi har valgt

![spørring](img/lagresporring.png)

* Her kan vi klippe og lime teksten inn i python
* Eller lagre som fil og åpne i python med json.load("filnavn.json"):
```python
import json
with open("filnavn.json", "r") as file:
    ssb_query = json.load(file)
#
#
```

* SSB-spørringen inneholder et felt "query", som er en liste av "dictionaries" med variablene eller feltene fra tabellen vi vil hente
* Hvert dataobjekt vi spør etter har under "selection" et felt "filter" hvor vi angir hvilke av dataene vi vil ha

### SSB: Filter
* Vi kan velge "item", da spesifiserer vi alle verdiene:
  * `"filter": "item", "values": [«Liste med verdiene vi vil ha»]`
* Eller "top", da ber vi om feks de første 10 verdiene
  * `"filter": "top", "values": [«antall»]`
* Eller "all", da ber vi om alle som passer inn i et gitt "mønster"
  * `"filter": "all", "values": ["*"]` Alle resultat
  * `"filter": "all", "values": ["198*"]` Alle årstall fra 80-tallet ("*" matcher med 0,1,2,3...)

In [ ]:
import requests
import json
import random

jsonstreng = """{ "query": [
    {"code": "ContentsCode","selection": {
        "filter": "item",
        "values": ["Konkursar"]
      }
    },
    {"code": "Tid", "selection": {
        "filter": "all",
        "values": []
      }
    }
  ],"response": {"format": "json-stat2"}
}"""



In [ ]:
#Spørring etter åpna konkurser på 80-tallet
ssb_query_string = """
{
  "query": [
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Konkursar"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "all",
        "values": ["198*"]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
"""

In [ ]:
#Spørring etter åpna konkurser de 15 siste månedene
ssb_query_string = """
{
  "query": [
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Konkursar"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "top",
        "values": ["15"]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
"""

# Eksempel: SSB-spørring->Pandas dataframe via pyjstat

In [ ]:
# Gå inn i api-konsoll eller statistikkbanken og lag en spørring og last ned spørringen som json fil
